
Using Activation Functions in the network.
* Step function - 0 or 1. Not granular enough
* Sigmoud - Gradual change from 0 to 1 output. Has a problem that makes
Rectified Linear looks better
* Rectified Linear - output is X when X > 0, else 0. Fast (very simple), works
well

Why even use activation? ==> Linear activation functions results in the output
being linear as well. We need some kind of nonlinear activation function. The
type of non-linear activation function is less important apparently. 



In [9]:
import numpy as np
import matplotlib.pyplot as plt
import nnfs
from nnfs.datasets import spiral_data
nnfs.init()
plt.style.use('dark_background')

In [10]:
class Layer_Dense:
    def __init__(self, n_inputs, n_nuerons):
        """
        Sets the initial weights and the biases for the given layer. This is a dense layer because all of 
        the output values are passed to the input values of the next layer
        n_input - expected number of inputs. Can be from inputs or previous layer
        n_nuerons - number of nuerons to use in the layer
        """
        self.weights = 0.10 * np.random.randn(n_inputs, n_nuerons)
        self.biases = np.zeros((1, n_nuerons))


    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases

In [11]:
class Activation_RelU:
    def __init__(self):
        """
        Activates the nuerons
        """
        pass
    
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)

In [12]:
class Activation_Softmax:
    def __init__(self):
        """
        Cleans up the output values so they are uniform without throwing out negative values.
        """
        pass
    
    def forward(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

In [13]:
class Loss:
    def __init__(self):
        """
        Calculates the data aand regularization losses given model output 
        and ground truth values. This is a common class that will be inherited by later loss classes
        to build upon
        """
        pass
    def calculate(self, output, y):
        """
        output: output of the model
        y: ground truth values
        """
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss
    
class Loss_CategoricalCrossEntropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        # clip to prevent log(0) error. Both sides to prevent value biasing
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)
        
        # Checking for categorical labels vs one-hot encoded
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[
            range(samples),
            y_true
            ]
            # Mask values - only for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(
            y_pred_clipped*y_true,
            axis=1
            )
        
        negative_log = -np.log(correct_confidences)
        return negative_log

In [14]:
########################### Now put it to work #############################

X, y = spiral_data(samples=100, classes=3)

In [15]:

dense1 = Layer_Dense(2,3)
activation1 = Activation_RelU()

dense2 = Layer_Dense(3, 3)
activation2 = Activation_Softmax()

dense1.forward(X)
activation1.forward(dense1.output)

dense2.forward(activation1.output)
activation2.forward(dense2.output)

print(activation2.output[:5])

[[0.33333334 0.33333334 0.33333334]
 [0.33331734 0.33331832 0.33336434]
 [0.3332888  0.33329153 0.33341965]
 [0.33325943 0.33326396 0.33347666]
 [0.33323312 0.33323926 0.33352762]]


In [16]:
## Actually using the loss values
loss_function = Loss_CategoricalCrossEntropy()
loss = loss_function.calculate(activation2.output, y)
print(loss)

1.098445


In [17]:
predictions = np.argmax(activation2.output, axis=1)
if len(y.shape) == 2:
    y = np.argmax(y, axis=1)
accuracy = np.mean(predictions == y)
print(f"Accuracy: {accuracy}")

Accuracy: 0.34
